In [28]:
# from sklearn.datasets import make_swiss_roll
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from tensorflow import keras  
from keras.models import Model
from keras import Input
from keras.layers import Dense
from keras.optimizers import Adam
import numpy as np


AttributeError: module 'keras.api._v2.keras' has no attribute 'engine'

In [31]:
from vanilla_ae_functions import *

AttributeError: module 'keras.api._v2.keras' has no attribute 'engine'

In [30]:
from tensorflow.python.keras.engine.keras_tensor import KerasTensor
import os

In [6]:
# Split the dataset into training and remaining data
# X_train, X_remaining = train_test_split(subset_vectors, test_size=0.4, random_state=42)

# Split the remaining data into validation and testing sets
# X_val, X_test = train_test_split(X_remaining, test_size=0.5, random_state=42)


In [3]:
# Specify the path to your Word2Vec dataset
dataset_path = "GoogleNews-vectors-negative300.bin.gz"

# Load the Word2Vec model
model = KeyedVectors.load_word2vec_format(dataset_path, binary=True)

In [4]:
all_vectors = model.vectors

In [5]:
X_train_unnormalised = all_vectors[0:10000]

In [1]:
def load_input(dataset_path: str) -> np.ndarray:
    model = KeyedVectors.load_word2vec_format(dataset_path, binary=True)
    all_vectors = model.vectors
    return all_vectors

In [ ]:
# in between offer cut down of data

In [5]:
all_vectors = load_input("GoogleNews-vectors-negative300.bin.gz")[0:10000]

In [3]:
def normalise_abs_input(unnormalised_dataset: np.ndarray) -> np.ndarray:
    # Figure out the min value, so to make all vectors positive
    min_x_unormalised = min([min(i) for i in unnormalised_dataset])
    pos_x_train = unnormalised_dataset+abs(min_x_unormalised)
    
    # Normalise the data by dividing by the max of the positive data
    max_pos_x = max([max(i) for i in pos_x_train])
    x_train = pos_x_train/max_pos_x
    return x_train

In [6]:
x_train = normalise_abs_input(all_vectors)

In [19]:
def vanilla_ae(input: KerasTensor, x_train: np.ndarray, original_dim: int = 300, hl_dim: int = 2, epochs: int = 25, batch_size: int = 128):
    encoded = Dense(hl_dim, activation='relu')(input)
    decoded = Dense(original_dim, activation='relu')(encoded)
    
    autoencoder = Model(input, decoded)

    autoencoder.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.001))
    autoencoder.fit(x_train, x_train, epochs=epochs, batch_size=batch_size)

    return autoencoder

### Test 1: 1 hidden layers, size = 2
reference: https://stats.stackexchange.com/questions/345208/optimal-hidden-units-size

In [12]:
# number of neurons at the input layer
original_dim = 300  
# hidden layer dimension
hl_dim = 2

In [13]:
input = Input(shape=(original_dim,), name='Encoder_Input_Layer')

In [15]:
vanilla_ae(input)

Epoch 1/25
79/79 [==============================] - 2s 4ms/step - loss: 4.5512
Epoch 2/25
79/79 [==============================] - 0s 6ms/step - loss: 4.4206
Epoch 3/25
79/79 [==============================] - 1s 6ms/step - loss: 4.4093
Epoch 4/25
79/79 [==============================] - 0s 6ms/step - loss: 4.4054
Epoch 5/25
79/79 [==============================] - 0s 6ms/step - loss: 4.4034
Epoch 6/25
79/79 [==============================] - 1s 6ms/step - loss: 4.4021
Epoch 7/25
79/79 [==============================] - 1s 6ms/step - loss: 4.4011
Epoch 8/25
79/79 [==============================] - 0s 6ms/step - loss: 4.4005
Epoch 9/25
79/79 [==============================] - 0s 5ms/step - loss: 4.3999
Epoch 10/25
79/79 [==============================] - 0s 6ms/step - loss: 4.3995
Epoch 11/25
79/79 [==============================] - 0s 5ms/step - loss: 4.3992
Epoch 12/25
79/79 [==============================] - 0s 5ms/step - loss: 4.3990
Epoch 13/25
79/79 [==============================

In [23]:
# latent_inputs = Input(shape=(latent_dim,), name='Input_Z_Sampling')
latent_inputs = Input(shape=(hl_dim,), name='Input_Z_Sampling')

##### Given that it is a multilayer perceptron, RELU Activation function
reference: https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/ 

know it is a MLP, as:
- MLP : MLP is a basic feedforward neural network where information flows from the input layer through one or more hidden layers to the output layer. classification and regression tasks
- CNN : processing grid-like data, such as images or audio spectrograms, learning local patterns and features from input data
- RNN : process sequential or time-series data, where the order of data points matters. 

In [20]:
encoded = Dense(hl_dim, activation='relu')(input)
decoded = Dense(original_dim, activation='relu')(encoded)  # Adjusted to match input dimension
autoencoder = Model(input, decoded)
autoencoder.compile(loss="binary_crossentropy", 
                    optimizer=Adam(learning_rate=0.001),
                    metrics=["accuracy"])
autoencoder.fit(x_train, x_train, epochs=10, batch_size=128)

Epoch 1/10
79/79 [==============================] - 1s 3ms/step - loss: 8.1067 - accuracy: 0.0028
Epoch 2/10
79/79 [==============================] - 0s 3ms/step - loss: 8.1067 - accuracy: 0.0028
Epoch 3/10
79/79 [==============================] - 0s 5ms/step - loss: 8.1067 - accuracy: 0.0028
Epoch 4/10
79/79 [==============================] - 0s 3ms/step - loss: 8.1067 - accuracy: 0.0028
Epoch 5/10
79/79 [==============================] - 0s 3ms/step - loss: 8.1067 - accuracy: 0.0028
Epoch 6/10
79/79 [==============================] - 0s 4ms/step - loss: 8.1067 - accuracy: 0.0028
Epoch 7/10
79/79 [==============================] - 0s 3ms/step - loss: 8.1067 - accuracy: 0.0028
Epoch 8/10
79/79 [==============================] - 0s 3ms/step - loss: 8.1067 - accuracy: 0.0028
Epoch 9/10
79/79 [==============================] - 0s 4ms/step - loss: 8.1067 - accuracy: 0.0028
Epoch 10/10
79/79 [==============================] - 0s 4ms/step - loss: 8.1067 - accuracy: 0.0028


In [26]:
import matplotlib.pyplot as plt

In [25]:
encoded_data = autoencoder.predict(x_train)

313/313 [==============================] - 1s 2ms/step


In [26]:
# Measuring the error 
mse = np.mean(np.square(x_train - encoded_data))
print(mse)

0.1493772


In [27]:
# Determining the accuracy doesn't really make sense
acc = np.mean(x_train == encoded_data)
print(acc)
# Cross validation during testing
# Runtime 

1e-06
